# Import python dataset

In [ ]:
notebookName = 'Different_Field.ipynb'

In [ ]:
k = 1

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import pandas as pd
import time
import functools


import numpy as np
import functools
import time

import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.optimizers import Adam
from tensorflow import reduce_mean
import gc
from sklearn.preprocessing import *



In [ ]:
os.makedirs('./output/Figures/Individual')
os.makedirs('./output/Figures/Result')
os.makedirs('./output/Figures/Prediction')
os.makedirs('./output/Figures/Different_Field')
os.makedirs('./output/Figures/Different_Field/Old')
os.makedirs('./output/Figures/Prediction/Old')
os.makedirs('./ganmodedls')
os.makedirs('./logs/reconstrunction_log_dir')
os.makedirs('./logs/generator')
os.makedirs('./logs/discriminator')
os.makedirs('./output/Figures/Loss')

## Load data

In [ ]:
n_features = 7

In [ ]:
data01= np.loadtxt('../input/pollutant/01.txt') 
print(data01.shape)

data02= np.loadtxt('../input/pollutant/02.txt') 
print(data02.shape)

data04= np.loadtxt('../input/pollutant/04.txt') 
print(data04.shape)


groups = ['PM10','PM2_5', 'PM1', 'CO2' ,'Temeprature' ,'RH']
groups = np.array(groups)
print(groups.shape)

with open("../input/pollutant/10_2_time.csv", "r", encoding="utf-8") as fr:
    times = [fr.readlines()]
    
times = np.array(times)
times = times.T

print('times = ',times[:,0].shape)
print(times[0,:])

# Preparing Dataset

In [ ]:
codings_size = n_features

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=[-1,1])
X_train_scaled = scaler.fit_transform(data01)

X_train_scaled.shape 


In [ ]:
#scaler inverse_transform
X_train = scaler.inverse_transform(((X_train_scaled)))
X_train.shape 

# Train the GAN

In [ ]:
ae = keras.models.load_model('../input/ganmodels/ae10000.h5').layers
encoder, decoder = keras.models.load_model('../input/ganmodels/ae10000.h5').layers

### Preparing the training dataset

In [ ]:
n = 0

In [ ]:
def concat_timesteps(X_train_scaled, ntimes, step):
    X_train_concat = []
    for i in range(len(X_train_scaled) - ntimes*step):
        X_train_concat.append(X_train_scaled[i:i+ntimes*step:step])
    return np.array(X_train_concat)

ntimes = 100 # Consecutive times for the GAN
step = 1 # step between times (1 mins)

X_train_concat = concat_timesteps(X_train_scaled, ntimes, step)#X_train_scaled
X_train_concat = X_train_concat.reshape(X_train_concat.shape[0],ntimes,n_features,1)
print(X_train_concat.shape)


In [ ]:
BATCH_SIZE = 256
latent_space = 50

# Create a tensorflow dataset and split it into batches
train_dataset = X_train_concat.astype('float32').reshape(X_train_concat.shape[0], ntimes, n_features, 1).astype('float32')
train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset)
train_dataset = train_dataset.shuffle(len(X_train_scaled))
train_dataset = train_dataset.batch(BATCH_SIZE)

for data in train_dataset:
    print(data.shape)
    break

# Predict with the GAN

In [ ]:
mse = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(1e-1) #1e-4

def mse_loss(inp, outp):
    inp = tf.reshape(inp, [(ntimes-1), 1])
    outp = tf.reshape(outp, [(ntimes-1), 1])
    print("input:",inp.shape)
    print("output:",outp.shape)
    
    return mse(inp, outp)


def optimize_coding(latent_values, real_coding, epochs, control_variable):
    
    for epoch in range(epochs):
        opt_step(latent_values, real_coding, control_variable).numpy()
        

    return latent_values  #returns the optimized input that generates the desired output


#@tf.function
def opt_step(latent_values, real_coding, control_variable):
    with tf.GradientTape() as tape:
        tape.watch(latent_values)
        gen_output = decoder(latent_values, training=False)  #results from generator
        loss = mse_loss(real_coding, gen_output[-1,:(ntimes - 1),control_variable])
        #If training for 9 time levels, it finds the loss between the first 8 outputs from the generator and 8 real outputs

    gradient = tape.gradient(loss, latent_values)   #gradient of the loss ws to the input
    optimizer.apply_gradients(zip([gradient], [latent_values]))   #applies gradients to the input
    
    return loss

## PM10 

In [ ]:
n = 500
control_variable = 2 #PM10
epochs = 2000
predict_points = 1 #X_train.shape[0]-n-ntimes

In [ ]:
np.random.seed(0)
tf.random.set_seed(0)

real_coding = X_train_concat[n,:,control_variable,:]
real_coding = real_coding.reshape(1,-1)
real_coding = real_coding[:, :(ntimes - 1)]
real_coding = tf.constant(real_coding)
real_coding = tf.cast(real_coding, dtype=tf.float32)
#print('first real_coding ', real_coding.shape)

latent_values = tf.random.normal([len(real_coding), latent_space])  
latent_values = tf.Variable(latent_values)#make input a tensorflow variable so it can be trained
latent_values = optimize_coding(latent_values, real_coding, epochs, control_variable)

X_predict = list(decoder(latent_values).numpy().reshape(-1,n_features))
gen_predict = X_predict[-1]


#print(gen_predict)


real_coding = np.concatenate((real_coding, gen_predict.reshape(1,-1)), axis=1)[:,n_features:]
#print('second real_coding ', real_coding.shape)
real_coding = tf.constant(real_coding)
real_coding = tf.cast(real_coding, dtype=tf.float32)

for i in range(predict_points): #range(2000,len(X_train_concat)-1):
    latent_values = optimize_coding(latent_values, real_coding, epochs, control_variable)
    gen_predict = decoder(latent_values, training=False)[:,(ntimes - 1):,:].numpy()
    gen_predict = gen_predict.flatten()
    X_predict.append(gen_predict.flatten())
    
    real_coding = np.concatenate((real_coding, gen_predict.reshape(1,-1)), axis=1)[:,n_features:]
    #print('thrid real_coding ', real_coding.shape)
    real_coding = tf.constant(real_coding)
    real_coding = tf.cast(real_coding, dtype=tf.float32)
    
X_predict = np.array(X_predict)

In [ ]:
X_generated = scaler.inverse_transform((X_predict))
X_generated = X_generated.reshape(len(X_predict), n_features)
X_generated[X_generated<0] = 0 
np.save('X_generatedall.npy',X_generated)
                                       

fig, ax = plt.subplots(2,4, figsize=[20,10])
for i, group in enumerate(groups):
    #ax.flatten()[i].plot(X_generated[:,i+1], '-',c='orange', label='generated')
    ax.flatten()[i].plot(X_train[n:(n+ntimes+predict_points),i+1], '-', label='real')
    ax.flatten()[i].plot(X_generated[:,i+1], '-',c='orange', label='generated')
    #ax.flatten()[i].plot(X_real[:,i], '-o', label='real2')
    ax.flatten()[i].set_title(group)
    ax.flatten()[i].legend()
    
plt.savefig('./output/Figures/Different_Field/Old/PM10_entire'+'_v'+str(k)+'_epoch_'+str(epochs)+'.png')

## CO2 

In [ ]:
control_variable = 1 #CO2

In [ ]:
np.random.seed(0)
tf.random.set_seed(0)

real_coding = X_train_concat[n,:,control_variable,:]
real_coding = real_coding.reshape(1,-1)
real_coding = real_coding[:, :(ntimes - 1)]
real_coding = tf.constant(real_coding)
real_coding = tf.cast(real_coding, dtype=tf.float32)
#print('first real_coding ', real_coding.shape)

latent_values = tf.random.normal([len(real_coding), latent_space])  
latent_values = tf.Variable(latent_values)#make input a tensorflow variable so it can be trained
latent_values = optimize_coding(latent_values, real_coding, epochs, control_variable)

X_predict = list(decoder(latent_values).numpy().reshape(-1,n_features))
gen_predict = X_predict[-1]


#print(gen_predict)


real_coding = np.concatenate((real_coding, gen_predict.reshape(1,-1)), axis=1)[:,n_features:]
#print('second real_coding ', real_coding.shape)
real_coding = tf.constant(real_coding)
real_coding = tf.cast(real_coding, dtype=tf.float32)

for i in range(predict_points): #range(2000,len(X_train_concat)-1):
    latent_values = optimize_coding(latent_values, real_coding, epochs, control_variable)
    gen_predict = decoder(latent_values, training=False)[:,(ntimes - 1):,:].numpy()
    gen_predict = gen_predict.flatten()
    X_predict.append(gen_predict.flatten())
    
    real_coding = np.concatenate((real_coding, gen_predict.reshape(1,-1)), axis=1)[:,n_features:]
    #print('thrid real_coding ', real_coding.shape)
    real_coding = tf.constant(real_coding)
    real_coding = tf.cast(real_coding, dtype=tf.float32)
    
X_predict = np.array(X_predict)




In [ ]:
X_generated = scaler.inverse_transform((X_predict))
X_generated = X_generated.reshape(len(X_predict), n_features)
X_generated[X_generated<0] = 0 
np.save('X_generatedall.npy',X_generated)
                                       

fig, ax = plt.subplots(2,4, figsize=[20,10])
for i, group in enumerate(groups):
    #ax.flatten()[i].plot(X_generated[:,i+1], '-',c='orange', label='generated')
    ax.flatten()[i].plot(X_train[n:(n+ntimes+predict_points),i+1], '-', label='real')
    ax.flatten()[i].plot(X_generated[:,i+1], '-',c='orange', label='generated')
    #ax.flatten()[i].plot(X_real[:,i], '-o', label='real2')
    ax.flatten()[i].set_title(group)
    ax.flatten()[i].legend()
    
plt.savefig('./output/Figures/Different_Field/Old/CO2_real_30mins_'+'_v'+str(k)+'_epoch_'+str(epochs)+'.png')

## RH

In [ ]:
control_variable = 5 #RH

In [ ]:
np.random.seed(0)
tf.random.set_seed(0)

real_coding = X_train_concat[n,:,control_variable,:]
real_coding = real_coding.reshape(1,-1)
real_coding = real_coding[:, :(ntimes - 1)]
real_coding = tf.constant(real_coding)
real_coding = tf.cast(real_coding, dtype=tf.float32)
#print('first real_coding ', real_coding.shape)

latent_values = tf.random.normal([len(real_coding), latent_space])  
latent_values = tf.Variable(latent_values)#make input a tensorflow variable so it can be trained
latent_values = optimize_coding(latent_values, real_coding, epochs, control_variable)

X_predict = list(decoder(latent_values).numpy().reshape(-1,n_features))
gen_predict = X_predict[-1]


#print(gen_predict)


real_coding = np.concatenate((real_coding, gen_predict.reshape(1,-1)), axis=1)[:,n_features:]
#print('second real_coding ', real_coding.shape)
real_coding = tf.constant(real_coding)
real_coding = tf.cast(real_coding, dtype=tf.float32)

for i in range(predict_points): #range(2000,len(X_train_concat)-1):
    latent_values = optimize_coding(latent_values, real_coding, epochs, control_variable)
    gen_predict = decoder(latent_values, training=False)[:,(ntimes - 1):,:].numpy()
    gen_predict = gen_predict.flatten()
    X_predict.append(gen_predict.flatten())
    
    real_coding = np.concatenate((real_coding, gen_predict.reshape(1,-1)), axis=1)[:,n_features:]
    #print('thrid real_coding ', real_coding.shape)
    real_coding = tf.constant(real_coding)
    real_coding = tf.cast(real_coding, dtype=tf.float32)
    
X_predict = np.array(X_predict)


In [ ]:
X_generated = scaler.inverse_transform((X_predict))
X_generated = X_generated.reshape(len(X_predict), n_features)
X_generated[X_generated<0] = 0 
np.save('X_generatedall.npy',X_generated)
                                       

fig, ax = plt.subplots(2,4, figsize=[20,10])
for i, group in enumerate(groups):
    #ax.flatten()[i].plot(X_generated[:,i], '-',c='orange', label='generated')
    ax.flatten()[i].plot(X_train[n:(n+ntimes+predict_points),i+1], '-', label='real')
    ax.flatten()[i].plot(X_generated[:,i+1], '-',c='orange', label='generated')
    #ax.flatten()[i].plot(X_real[:,i], '-o', label='real2')
    ax.flatten()[i].set_title(group)
    ax.flatten()[i].legend()
    
plt.savefig('./output/Figures/Different_Field/Old/RH_30mins_real'+'_v'+str(k)+'_epoch_'+str(epochs)+'.png')

## Temperature (30mins)

In [ ]:
control_variable = 6 #Temperature

In [ ]:
np.random.seed(0)
tf.random.set_seed(0)

real_coding = X_train_concat[n,:,control_variable,:]
real_coding = real_coding.reshape(1,-1)
real_coding = real_coding[:, :(ntimes - 1)]
real_coding = tf.constant(real_coding)
real_coding = tf.cast(real_coding, dtype=tf.float32)
#print('first real_coding ', real_coding.shape)

latent_values = tf.random.normal([len(real_coding), latent_space])  
latent_values = tf.Variable(latent_values)#make input a tensorflow variable so it can be trained
latent_values = optimize_coding(latent_values, real_coding, epochs, control_variable)

X_predict = list(decoder(latent_values).numpy().reshape(-1,n_features))
gen_predict = X_predict[-1]


#print(gen_predict)


real_coding = np.concatenate((real_coding, gen_predict.reshape(1,-1)), axis=1)[:,n_features:]
#print('second real_coding ', real_coding.shape)
real_coding = tf.constant(real_coding)
real_coding = tf.cast(real_coding, dtype=tf.float32)

for i in range(predict_points): #range(2000,len(X_train_concat)-1):
    latent_values = optimize_coding(latent_values, real_coding, epochs, control_variable)
    gen_predict = decoder(latent_values, training=False)[:,(ntimes - 1):,:].numpy()
    gen_predict = gen_predict.flatten()
    X_predict.append(gen_predict.flatten())
    
    real_coding = np.concatenate((real_coding, gen_predict.reshape(1,-1)), axis=1)[:,n_features:]
    #print('thrid real_coding ', real_coding.shape)
    real_coding = tf.constant(real_coding)
    real_coding = tf.cast(real_coding, dtype=tf.float32)
    
X_predict = np.array(X_predict)



In [ ]:
X_generated = scaler.inverse_transform((X_predict))
X_generated = X_generated.reshape(len(X_predict), n_features)
X_generated[X_generated<0] = 0 
np.save('X_generatedall.npy',X_generated)
                                       

fig, ax = plt.subplots(2,4, figsize=[20,10])
for i, group in enumerate(groups):
   # ax.flatten()[i].plot(X_generated[:,i], '-',c='orange', label='generated')
    ax.flatten()[i].plot(X_train[n:(n+ntimes+predict_points),i+1], '-', label='real')
    ax.flatten()[i].plot(X_generated[:,i+1], '-',c='orange', label='generated')
    #ax.flatten()[i].plot(X_real[:,i], '-o', label='real2')
    ax.flatten()[i].set_title(group)
    ax.flatten()[i].legend()
    
plt.savefig('./output/Figures/Different_Field/Old/Temp_30mins_'+'_v'+str(k)+'_epoch_'+str(epochs)+'.png')

# PM 2.5

In [ ]:
control_variable = 3 

In [ ]:
np.random.seed(0)
tf.random.set_seed(0)

real_coding = X_train_concat[n,:,control_variable,:]
real_coding = real_coding.reshape(1,-1)
real_coding = real_coding[:, :(ntimes - 1)]
real_coding = tf.constant(real_coding)
real_coding = tf.cast(real_coding, dtype=tf.float32)
#print('first real_coding ', real_coding.shape)

latent_values = tf.random.normal([len(real_coding), latent_space])  
latent_values = tf.Variable(latent_values)#make input a tensorflow variable so it can be trained
latent_values = optimize_coding(latent_values, real_coding, epochs, control_variable)

X_predict = list(decoder(latent_values).numpy().reshape(-1,n_features))
gen_predict = X_predict[-1]


#print(gen_predict)


real_coding = np.concatenate((real_coding, gen_predict.reshape(1,-1)), axis=1)[:,n_features:]
#print('second real_coding ', real_coding.shape)
real_coding = tf.constant(real_coding)
real_coding = tf.cast(real_coding, dtype=tf.float32)

for i in range(predict_points): #range(2000,len(X_train_concat)-1):
    latent_values = optimize_coding(latent_values, real_coding, epochs, control_variable)
    gen_predict = decoder(latent_values, training=False)[:,(ntimes - 1):,:].numpy()
    gen_predict = gen_predict.flatten()
    X_predict.append(gen_predict.flatten())
    
    real_coding = np.concatenate((real_coding, gen_predict.reshape(1,-1)), axis=1)[:,n_features:]
    #print('thrid real_coding ', real_coding.shape)
    real_coding = tf.constant(real_coding)
    real_coding = tf.cast(real_coding, dtype=tf.float32)
    
X_predict = np.array(X_predict)


X_generated = scaler.inverse_transform((X_predict))
X_generated = X_generated.reshape(len(X_predict), n_features)
X_generated[X_generated<0] = 0 
np.save('X_generatedall.npy',X_generated)
                                       

fig, ax = plt.subplots(2,4, figsize=[20,10])
for i, group in enumerate(groups):
    #ax.flatten()[i].plot(X_generated[:,i], '-',c='orange', label='generated')
    ax.flatten()[i].plot(X_train[n:(n+ntimes+predict_points),i+1], '-', label='real')
    ax.flatten()[i].plot(X_generated[:,i+1], '-',c='orange', label='generated')
    #ax.flatten()[i].plot(X_real[:,i], '-o', label='real2')
    ax.flatten()[i].set_title(group)
    ax.flatten()[i].legend()
    
plt.savefig('./output/Figures/Different_Field/Old/PM2_5_30mins_'+'_v'+str(k)+'_epoch_'+str(epochs)+'.png')

# PM 1

In [ ]:
control_variable = 4 

In [ ]:
np.random.seed(0)
tf.random.set_seed(0)

real_coding = X_train_concat[n,:,control_variable,:]
real_coding = real_coding.reshape(1,-1)
real_coding = real_coding[:, :(ntimes - 1)]
real_coding = tf.constant(real_coding)
real_coding = tf.cast(real_coding, dtype=tf.float32)
#print('first real_coding ', real_coding.shape)

latent_values = tf.random.normal([len(real_coding), latent_space])  
latent_values = tf.Variable(latent_values)#make input a tensorflow variable so it can be trained
latent_values = optimize_coding(latent_values, real_coding, epochs, control_variable)

X_predict = list(decoder(latent_values).numpy().reshape(-1,n_features))
gen_predict = X_predict[-1]


#print(gen_predict)


real_coding = np.concatenate((real_coding, gen_predict.reshape(1,-1)), axis=1)[:,n_features:]
#print('second real_coding ', real_coding.shape)
real_coding = tf.constant(real_coding)
real_coding = tf.cast(real_coding, dtype=tf.float32)

for i in range(predict_points): #range(2000,len(X_train_concat)-1):
    latent_values = optimize_coding(latent_values, real_coding, epochs, control_variable)
    gen_predict = decoder(latent_values, training=False)[:,(ntimes - 1):,:].numpy()
    gen_predict = gen_predict.flatten()
    X_predict.append(gen_predict.flatten())
    
    real_coding = np.concatenate((real_coding, gen_predict.reshape(1,-1)), axis=1)[:,n_features:]
    #print('thrid real_coding ', real_coding.shape)
    real_coding = tf.constant(real_coding)
    real_coding = tf.cast(real_coding, dtype=tf.float32)
    
X_predict = np.array(X_predict)


In [ ]:
X_generated = scaler.inverse_transform((X_predict))
X_generated = X_generated.reshape(len(X_predict), n_features)
X_generated[X_generated<0] = 0 
np.save('X_generatedall.npy',X_generated)
                                       

fig, ax = plt.subplots(2,4, figsize=[20,10])
for i, group in enumerate(groups):
    #ax.flatten()[i].plot(X_generated[:,i+1], '-',c='orange', label='generated')
    ax.flatten()[i].plot(X_train[n:(n+ntimes+predict_points),i+1], '-', label='real')
    ax.flatten()[i].plot(X_generated[:,i+1], '-',c='orange', label='generated')
    #ax.flatten()[i].plot(X_real[:,i], '-o', label='real2')
    ax.flatten()[i].set_title(group)
    ax.flatten()[i].legend()
    
plt.savefig('./output/Figures/Different_Field/Old/PM1_30mins_'+'_v'+str(k)+'_epoch_'+str(epochs)+'.png')